In [83]:
import pandas as pd
from geopy.geocoders import Nominatim
import geopandas as gpd
# from tqdm import tqdm
from tqdm.auto import tqdm  # for notebooks
tqdm.pandas()
import re
import urllib.parse

## Geocode scraping result

In [22]:
df=pd.read_csv('mayor_offices_saved.csv')

In [55]:
def complement_address(address):
    # street types
    address=address.replace(' u.', ' utca')

    # general
    address=address.replace('.', '')
    
    # names
    address = address.replace('Petőfi S', 'Petőfi Sándor')
    address = address.replace('Ady E', 'Ady Endre')
    address = address.replace('Kossuth L', 'Kossuth Lajos')
    address = address.replace('Szent I', 'Szent István')
    address = address.replace('Arany J', 'Arany János')
    address = address.replace('Rákóczi F', 'Rákóczi Ferenc')
    address = address.replace('Zrínyi M', 'Zrínyi Miklós')
    address = address.replace('Béri-Balogh Á', 'Béri-Balogh Ádám')
    address = address.replace('József A', 'József Attila')
    address = address.replace('Bartók B', 'Bartók Béla')
    address = address.replace('Dózsa Gy', 'Dózsa György')
    return address

In [57]:
df['full_address']=df.address.apply(complement_address)
df['name']=df.name.apply(lambda x: x.replace(''))

In [143]:
get_muni_name=lambda href:urllib.parse.unquote(href.split('/')[2], encoding='latin-1')

In [147]:
df['muni_name']=df.href.apply(get_muni_name)

In [59]:
df

,href,name,address,full_address
0,/telepules/Aba,Polgármesteri Hivatal Aba,"8127 Aba, Rákóczi u. 12.","8127 Aba, Rákóczi utca 12"
1,/telepules/Ab%e1dszal%f3k,Polgármesteri Hivatal Abádszalók,"5241 Abádszalók, Deák Ferenc út 12.","5241 Abádszalók, Deák Ferenc út 12"
2,/telepules/Aba%fajalp%e1r,Polgármesteri Hivatal Abaújkér,"3882 Abaújalpár, Abaújalpár Petőfi S. u. 54.","3882 Abaújalpár, Abaújalpár Petőfi Sándor utca 54"
3,/telepules/Aba%fajalp%e1r,Polgármesteri Hivatal Abaújkér,"3882 Abaújalpár, Abaújkér Rákóczi utca 26.","3882 Abaújalpár, Abaújkér Rákóczi utca 26"
4,/telepules/Aba%fajlak,Polgármesteri Hivatal Gadna,"3815 Abaújlak, Abaújlak Fő utca 82.","3815 Abaújlak, Abaújlak Fő utca 82"
...,...,...,...,...
2745,/telepules/%d6rv%e9nyes,Polgármesteri Hivatal Balatonudvari,"8242 Örvényes, Ady E. u. 16.","8242 Örvényes, Ady Endre utca 16"
2746,/telepules/%d6rv%e9nyes,Polgármesteri Hivatal Balatonudvari,"8242 Örvényes, Fenyves u. 1.","8242 Örvényes, Fenyves utca 1"
2747,/telepules/%d6sk%fc,Polgármesteri Hivatal Öskü,"8191 Öskü, Szabadság tér 1.","8191 Öskü, Szabadság tér 1"
2748,/telepules/%d6ttev%e9ny,Polgármesteri Hivatal Öttevény,"9153 Öttevény, Fő u. 100.","9153 Öttevény, Fő utca 100"


In [49]:
def geocode_address(address):
    try:
        geolocator = Nominatim(user_agent="my_geocoder")
        location = geolocator.geocode(address)
        return (location.latitude, location.longitude)
    except Exception as e:
        print(f'{address} could not be geocoded, skipped to the next one.')
        return None

In [51]:
df.full_address.progress_apply(geocode_address)

  0%|          | 0/2750 [00:00<?, ?it/s]

5241 Abádszalók, Deák Ferenc út 12. could not be geocoded, skipped to the next one.
3882 Abaújalpár, Abaújalpár Petőfi Sándor utca 54. could not be geocoded, skipped to the next one.
3882 Abaújalpár, Abaújkér Rákóczi utca 26. could not be geocoded, skipped to the next one.
3815 Abaújlak, Gadna Fő út. 4. could not be geocoded, skipped to the next one.
3753 Abod, Abod Magyar út 42. could not be geocoded, skipped to the next one.
6076 Ágasegyháza, Szent I tér 1 could not be geocoded, skipped to the next one.
3759 Aggtelek, Aggtelek Kossuth Lajos utca 8. could not be geocoded, skipped to the next one.
3779 Alacska, Alacska Dózsa Gy. utca 7. could not be geocoded, skipped to the next one.
3353 Aldebrő, Arany J. út 1. could not be geocoded, skipped to the next one.
3717 Alsódobsza, Alsódobsza Rákóczi út 44. could not be geocoded, skipped to the next one.
7147 Alsónána, Kossuth Lajos utca 27. could not be geocoded, skipped to the next one.
8924 Alsónemesapáti, Alsónemesapáti Petőfi Sándoru.1.

KeyboardInterrupt: 